# ImapRetriever

- TODO: Make sure API reference link is correct.

This will help you get started with the Imap [retriever](/docs/concepts/retrievers). For detailed documentation of all ImapRetriever features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/retrievers/langchain_imap.retrievers.Imap.ImapRetriever.html).

### Integration details

TODO: Select one of the tables below, as appropriate.

1: Bring-your-own data (i.e., index and search a custom corpus of documents):

| Retriever | Self-host | Cloud offering | Package |
| :--- | :--- | :---: | :---: |
[ImapRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain-imap.retrievers.langchain_imap.ImapRetriever.html) | ❌ | ❌ | langchain-imap |

2: External index (e.g., constructed from Internet data or similar)):

| Retriever | Source | Package |
| :--- | :--- | :---: |
[ImapRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain-imap.retrievers.langchain_imap.ImapRetriever.html) | Source description | langchain-imap |

## Setup

- TODO: Update with relevant info.

If you want to get automated tracing from individual queries, you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [1]:
from pathlib import Path
import subprocess
import os

preload_dir = Path(os.getcwd()).parent / "tests" / "fixtures" / "preload"
log_path = Path(os.getcwd()).parent / "tests"  / "container.log"

# GreenMail configuration
env_vars = {
    "GREENMAIL_OPTS": " ".join([
        "-Dgreenmail.setup.test.all",
        "-Dgreenmail.users=test:test123@localhost",
        "-Dgreenmail.users.login=local_part",
        "-Dgreenmail.preload.dir=/preload",
        "-Dgreenmail.verbose",
        "-Dgreenmail.hostname=0.0.0.0"
    ])
}

# Prepare podman run command
container_name = "langchain-imap-test"
cmd = [
    "podman", "run", "--rm", "-d",
    "--name", container_name,
    "-e", f"GREENMAIL_OPTS={env_vars['GREENMAIL_OPTS']}",
    "-v", f"{preload_dir}:/preload:ro,Z",
    "-p", "3143:3143",
    "-p", "3993:3993",
    "-p", "8080:8080",
    "--log-driver", "k8s-file",
    "--log-opt", f"path={log_path.absolute()}",
    "docker.io/greenmail/standalone:2.1.5",
]

result = subprocess.run(cmd, capture_output=True, text=True, check=True)


### Installation

This retriever lives in the `langchain-imap` package:

In [ ]:
%pip install -qU langchain-imap

## Instantiation

Now we can instantiate our retriever:

In [3]:
from langchain_imap import ImapRetriever, ImapConfig
config = ImapConfig(
    host="localhost",
    port=3143,
    user="test",
    password="test123",
    ssl_mode="plain",
    verify_cert=False,
)
retriever = ImapRetriever(
    config = config, k=50
)

## Usage

In [ ]:
query = 'ALL'
retriever.invoke(query)

[Document(metadata={'message_id': '1', 'date': '2023-10-03T09:00:00+00:00', 'from': 'boss@example.com', 'to': 'test@localhost', 'subject': 'URGENT: Project Deadline Approaching'}, page_content='To: test@localhost\nFrom: boss@example.com\nSubject: URGENT: Project Deadline Approaching\nCC: \nBCC: \nDate: 2023-10-03T09:00:00+00:00\nBody: <!--\ntitle: URGENT: Project Deadline\n-->\n\n# URGENT: Project Deadline Approaching!\n\nThe deadline for our major project is **approaching** quickly.\n\nPlease complete all assigned tasks *immediately*.\n\n* Task 1: Finish coding\n* Task 2: Write documentation\n* Task 3: Test functionality\n\nContact me if you need support.\n\n**Project Manager**\n\n'),
 Document(metadata={'message_id': '2', 'date': '2023-09-29T16:45:00+00:00', 'from': 'alice@example.com', 'to': 'test@localhost', 'subject': 'Team Meeting Notes'}, page_content="To: test@localhost\nFrom: alice@example.com\nSubject: Team Meeting Notes\nCC: \nBCC: \nDate: 2023-09-29T16:45:00+00:00\nBody: He

In [5]:
query = 'SUBJECT "URGENT"'
retriever.invoke(query)

[Document(metadata={'message_id': '1', 'date': '2023-10-03T09:00:00+00:00', 'from': 'boss@example.com', 'to': 'test@localhost', 'subject': 'URGENT: Project Deadline Approaching'}, page_content='To: test@localhost\nFrom: boss@example.com\nSubject: URGENT: Project Deadline Approaching\nCC: \nBCC: \nDate: 2023-10-03T09:00:00+00:00\nBody: <!--\ntitle: URGENT: Project Deadline\n-->\n\n# URGENT: Project Deadline Approaching!\n\nThe deadline for our major project is **approaching** quickly.\n\nPlease complete all assigned tasks *immediately*.\n\n* Task 1: Finish coding\n* Task 2: Write documentation\n* Task 3: Test functionality\n\nContact me if you need support.\n\n**Project Manager**\n\n'),
 Document(metadata={'message_id': '3', 'date': '2023-10-02T12:00:00+00:00', 'from': 'sender@example.com', 'to': 'test@localhost', 'subject': 'URGENT: System Maintenance Required'}, page_content='To: test@localhost\nFrom: sender@example.com\nSubject: URGENT: System Maintenance Required\nCC: \nBCC: \nDate:

## Use within a chain

Like other retrievers, ImapRetriever can be incorporated into LLM applications via [chains](/docs/how_to/sequence/).

We will need a LLM or chat model:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [24]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_imap import ImapRetriever, ImapConfig

if openai_api_key is None: 
    os.getenv("OPENROUTER_API_KEY")

# Setup OpenRouter LLM
llm = ChatOpenAI(
    model="google/gemini-2.5-flash-preview-09-2025",
    temperature=0,
    openai_api_key=openai_api_key,
    openai_api_base="https://openrouter.ai/api/v1"
)

# IMAP query generation prompt
query_prompt = ChatPromptTemplate.from_template(
    """Convert the following user question into an IMAP search query.
    
IMAP query syntax examples:
- 'FROM "john@example.com"' - emails from specific sender
- 'SUBJECT "project update"' - emails with specific subject
- 'SENTSINCE "01-Oct-2024"' - emails since specific date
- 'BODY "meeting"' - emails containing specific word in body
- 'FROM "boss@company.com" SUBJECT "urgent"' - combine criteria

IMPORTANT: Include only VALID imap command in output.
IMPORTANT: Do not include any other text in output.

User Question: {question}

IMAP Query:"""
)

# Answer generation prompt
answer_prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided from emails.

Context:
{context}

Question: {question}

Answer:"""
)

# IMAP retriever configuration
config = ImapConfig(
        host="localhost",
        port=3993,
        user="test",
        password="test123",
        ssl_mode="ssl",
        auth_method="login",
        verify_cert=False,
    )

retriever = ImapRetriever(
    config=config,
    k=5,
    attachment_mode="names_only"
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the chain
query_chain = query_prompt | llm | StrOutputParser()

def generate_imap_query(question):
    return query_chain.invoke({"question": question})

def search_emails(query):
    return retriever.invoke(query)

full_chain = (
    {
        "question": lambda x: x,
        "imap_query": lambda x: generate_imap_query(x)
    }
    | RunnablePassthrough.assign(
        context=lambda x: format_docs(search_emails(x["imap_query"]))
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)



In [26]:
TODO = full_chain.invoke("Please make a TODO based on the e-mails having URGENT in subject")

print(TODO)

TODO:
* Finish coding
* Write documentation
* Test functionality
* Perform system maintenance
* Update system


## Teardown Env

In [28]:
cmd = ["podman", "rm", "--force", "langchain-imap-test"]
result = subprocess.run(cmd, capture_output=True, text=True, check=True)

In [29]:
print(result)

CompletedProcess(args=['podman', 'rm', '--force', 'langchain-imap-test'], returncode=0, stdout='langchain-imap-test\n', stderr='time="2025-10-13T07:40:16+02:00" level=warning msg="StopSignal SIGTERM failed to stop container langchain-imap-test in 10 seconds, resorting to SIGKILL"\n')


## API reference

For detailed documentation of all ImapRetriever features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/retrievers/langchain_imap.retrievers.Imap.ImapRetriever.html).